#Capstone Project Notebook


In [65]:
import pandas as pd
import numpy as np
import urllib3
from bs4 import BeautifulSoup
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


In [66]:
!pip  install urllib2

ERROR: Could not find a version that satisfies the requirement urllib2 (from versions: none)
ERROR: No matching distribution found for urllib2


In [ ]:
#https://api.foursquare.com/v2/
#client ID
#client secret 
#version

##Scraping data from Wikipedia



In [67]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
http = urllib3.PoolManager()
page = http.request('GET',url)
soup = BeautifulSoup(page.data, 'html.parser')


/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [68]:
table_data = []
for tr in soup.find_all('tr')[2:]:
  tds = tr.find_all('td')
  try:
    table_data.append([(tds[0].text)[:-1],(tds[1].text)[:-1],(tds[2].text)[:-1]])
  except:
    print("index out of range")
  

index out of range


##Dataframe From Scrapped Data


In [69]:
#table_data[0:5]
df = pd.DataFrame(table_data,columns = ['PostalCode','Borough','Neighborhood'])
df.head()

,PostalCode,Borough,Neighborhood
0,M2A,Not assigned,Not assigned
1,M3A,North York,Parkwoods
2,M4A,North York,Victoria Village
3,M5A,Downtown Toronto,"Regent Park, Harbourfront"
4,M6A,North York,"Lawrence Manor, Lawrence Heights"


##removing Cells which contains "Not Assigned" in Brorough Column


In [70]:
df.drop(df[df['Borough'] == 'Not assigned'].index,inplace=True)

In [71]:
df.head()

,PostalCode,Borough,Neighborhood
1,M3A,North York,Parkwoods
2,M4A,North York,Victoria Village
3,M5A,Downtown Toronto,"Regent Park, Harbourfront"
4,M6A,North York,"Lawrence Manor, Lawrence Heights"
5,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


##Grouping the neighbourhood

In [72]:
df_neighbourhood = df.groupby(['PostalCode','Borough'],as_index=False).agg(lambda x: " ,".join(x))

df_neighbourhood.drop([0,1],inplace = True)
df_neighbourhood.head()

,PostalCode,Borough,Neighborhood
2,M1B,Scarborough,"Malvern, Rouge"
3,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
4,M1E,Scarborough,"Guildwood, Morningside, West Hill"
5,M1G,Scarborough,Woburn
6,M1H,Scarborough,Cedarbrae


In [73]:
for index, row in df_neighbourhood.iterrows():
  if row["Neighborhood"] == "Not assigned":
    row["Neighbourhood"] = row["Borough"]
df_neighbourhood.head()

,PostalCode,Borough,Neighborhood
2,M1B,Scarborough,"Malvern, Rouge"
3,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
4,M1E,Scarborough,"Guildwood, Morningside, West Hill"
5,M1G,Scarborough,Woburn
6,M1H,Scarborough,Cedarbrae


In [74]:
df_neighbourhood.shape

(104, 3)

##Mapping longitude and lattitude with all the neighbourhood

In [77]:
coordinates = pd.read_csv("https://cocl.us/Geospatial_data")
coordinates.rename(columns={"Postal Code":"PostalCode"},inplace=True)
coordinates.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [79]:
df_neighbourhood_new = df_neighbourhood.merge(coordinates,on = "PostalCode",how='left')
df_neighbourhood_new.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [80]:
column_names = ["PostalCode", "Borough", "Neighborhood", "Latitude", "Longitude"]
test_df = pd.DataFrame(columns=column_names)

test_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in test_list:
    test_df = test_df.append(df_neighbourhood_new[df_neighbourhood_new["PostalCode"]==postcode], ignore_index=True)
    
test_df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
1,M2H,North York,Hillcrest Village,43.803762,-79.363452
2,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
3,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
4,M4G,East York,Leaside,43.709060,-79.363452
5,M4M,East Toronto,Studio District,43.659526,-79.340923
6,M1R,Scarborough,"Wexford, Maryvale",43.750072,-79.295849
7,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437
8,M9L,North York,Humber Summit,43.756303,-79.565963
9,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har...",43.628947,-79.394420
